In [1]:
# import autoreload
# ?autoreload

%load_ext autoreload
%autoreload 2
import torch
from transformers import BertModel

In [2]:
model = BertModel.from_pretrained("prajjwal1/bert-tiny",
                                  # add_pooling_layer=False,
                                  torchscript=True)

inputs = torch.randint(1000, size=(1, 100)).long()
# model()
# with torch.onnx.select_model_mode_for_export(model, False):
  # trace, _ = torch.jit._get_trace_graph(model, args=(inputs,))
#     trace = torch.jit.trace(model, (inputs, ))
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 128, padding_idx=0)
    (position_embeddings): Embedding(512, 128)
    (token_type_embeddings): Embedding(2, 128)
    (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=128, out_features=128, bias=True)
            (key): Linear(in_features=128, out_features=128, bias=True)
            (value): Linear(in_features=128, out_features=128, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=128, out_features=128, bias=True)
            (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [3]:
config=model.config
config.hidden_act = 'gelu_fast'
config

BertConfig {
  "_name_or_path": "prajjwal1/bert-tiny",
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu_fast",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 128,
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 2,
  "num_hidden_layers": 2,
  "pad_token_id": 0,
  "torchscript": true,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

In [4]:
from torch import nn
from transformers.activations import ACT2FN

class BertIntermediate(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(config.hidden_size, config.intermediate_size)
        if isinstance(config.hidden_act, str):
            self.intermediate_act_fn = ACT2FN[config.hidden_act]
        else:
            self.intermediate_act_fn = config.hidden_act

    def forward(self, hidden_states):
        hidden_states = self.dense(hidden_states)
#         hidden_states = self.intermediate_act_fn(hidden_states)
        return hidden_states

In [5]:
fc_model = BertIntermediate(config)
input_len = 32
input_states = torch.rand((input_len, config.hidden_size))
fc_model.eval()

BertIntermediate(
  (dense): Linear(in_features=128, out_features=512, bias=True)
)

In [6]:
fc_trace = torch.jit.trace(fc_model, input_states)
fc_graph = fc_trace.inlined_graph

In [7]:
fc_graph

graph(%self.1 : __torch__.BertIntermediate,
      %input : Float(32:128, 128:1)):
  %2 : __torch__.torch.nn.modules.linear.Linear = prim::GetAttr[name="dense"](%self.1)
  %4 : int = prim::Constant[value=1](), scope: __module.dense # /Users/qqcao/.pyenv/versions/nlpnrg/lib/python3.8/site-packages/torch/nn/functional.py:1674:0
  %5 : Tensor = prim::GetAttr[name="bias"](%2)
  %6 : Tensor = prim::GetAttr[name="weight"](%2)
  %7 : Float(128:1, 512:128) = aten::t(%6), scope: __module.dense # /Users/qqcao/.pyenv/versions/nlpnrg/lib/python3.8/site-packages/torch/nn/functional.py:1674:0
  %8 : Float(32:512, 512:1) = aten::addmm(%5, %input, %7, %4, %4), scope: __module.dense # /Users/qqcao/.pyenv/versions/nlpnrg/lib/python3.8/site-packages/torch/nn/functional.py:1674:0
  return (%8)

In [8]:
list(fc_graph.nodes())

[%2 : __torch__.torch.nn.modules.linear.Linear = prim::GetAttr[name="dense"](%self.1),
 %4 : int = prim::Constant[value=1](), scope: __module.dense # /Users/qqcao/.pyenv/versions/nlpnrg/lib/python3.8/site-packages/torch/nn/functional.py:1674:0,
 %5 : Tensor = prim::GetAttr[name="bias"](%2),
 %6 : Tensor = prim::GetAttr[name="weight"](%2),
 %7 : Float(128:1, 512:128) = aten::t(%6), scope: __module.dense # /Users/qqcao/.pyenv/versions/nlpnrg/lib/python3.8/site-packages/torch/nn/functional.py:1674:0,
 %8 : Float(32:512, 512:1) = aten::addmm(%5, %input, %7, %4, %4), scope: __module.dense # /Users/qqcao/.pyenv/versions/nlpnrg/lib/python3.8/site-packages/torch/nn/functional.py:1674:0]

In [9]:
fc_in_nodes=list(fc_graph.inputs())
[i for i in dir(fc_in_nodes[0]) if not i.startswith('__')]

['copyMetadata',
 'debugName',
 'inferTypeFrom',
 'isCompleteTensor',
 'node',
 'offset',
 'replaceAllUsesWith',
 'requiresGrad',
 'requires_grad',
 'setDebugName',
 'setType',
 'setTypeAs',
 'toIValue',
 'type',
 'unique',
 'uses']

In [10]:
fc_in_nodes[1].debugName()

'input'

In [11]:
list(fc_graph.outputs())

[8 defined in (%8 : Float(32:512, 512:1) = aten::addmm(%5, %input, %7, %4, %4), scope: __module.dense # /Users/qqcao/.pyenv/versions/nlpnrg/lib/python3.8/site-packages/torch/nn/functional.py:1674:0
 )]

In [14]:
trace = torch.jit.trace(model, inputs)
graph = trace.inlined_graph

/Users/qqcao/work/transformers/src/transformers/modeling_utils.py:1669: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  input_tensor.shape == tensor_shape for input_tensor in input_tensors


In [ ]:
graph

In [15]:
ni=list(n.inputs())
ni

graph(%self.1 : __torch__.transformers.modeling_bert.___torch_mangle_78.BertModel,
      %input_ids : Long(1:100, 100:1)):
  %2 : __torch__.transformers.modeling_bert.___torch_mangle_77.BertPooler = prim::GetAttr[name="pooler"](%self.1)
  %3 : __torch__.transformers.modeling_bert.___torch_mangle_74.BertEncoder = prim::GetAttr[name="encoder"](%self.1)
  %4 : __torch__.transformers.modeling_bert.___torch_mangle_38.BertEmbeddings = prim::GetAttr[name="embeddings"](%self.1)
  %5 : int = prim::Constant[value=0]() # /Users/qqcao/work/transformers/src/transformers/modeling_bert.py:795:0
  %6 : int = aten::size(%input_ids, %5) # /Users/qqcao/work/transformers/src/transformers/modeling_bert.py:795:0
  %7 : Long() = prim::NumToTensor(%6)
  %8 : int = aten::Int(%7)
  %9 : int = aten::Int(%7)
  %10 : int = prim::Constant[value=1]() # /Users/qqcao/work/transformers/src/transformers/modeling_bert.py:795:0
  %11 : int = aten::size(%input_ids, %10) # /Users/qqcao/work/transformers/src/transformers/mod

In [112]:
ni=list(n.inputs())
ni

[5 defined in (%5 : Tensor = prim::GetAttr[name="bias"](%2)
 ),
 input defined in (%self.1 : __torch__.___torch_mangle_3.BertIntermediate, %input : Float(32:128, 128:1) = prim::Param()
 ),
 7 defined in (%7 : Float(128:1, 512:128) = aten::t(%6), scope: __module.dense # /Users/qqcao/.pyenv/versions/nlpnrg/lib/python3.8/site-packages/torch/nn/functional.py:1674:0
 ),
 4 defined in (%4 : int = prim::Constant[value=1](), scope: __module.dense # /Users/qqcao/.pyenv/versions/nlpnrg/lib/python3.8/site-packages/torch/nn/functional.py:1674:0
 ),
 4 defined in (%4 : int = prim::Constant[value=1](), scope: __module.dense # /Users/qqcao/.pyenv/versions/nlpnrg/lib/python3.8/site-packages/torch/nn/functional.py:1674:0
 )]

In [146]:
type(ni[4].type())

torch._C.IntType

In [153]:
ni[1].type().sizes()

[32, 128]

In [142]:
a is None

True

In [137]:
isinstance(ni[1].type(), torch.TensorType)

True

In [81]:
n.kind()

'aten::addmm'

In [98]:
n.output().debugName()

'8'

In [104]:
n.output().type()

Tensor

In [102]:
n.scopeName()

'__module.dense'

In [108]:
nodes[0].kind()

'prim::GetAttr'

In [101]:
list(n.inputs())

[5 defined in (%5 : Tensor = prim::GetAttr[name="bias"](%2)
 ),
 input defined in (%self.1 : __torch__.___torch_mangle_3.BertIntermediate, %input : Float(32:128, 128:1) = prim::Param()
 ),
 7 defined in (%7 : Float(128:1, 512:128) = aten::t(%6), scope: __module.dense # /Users/qqcao/.pyenv/versions/nlpnrg/lib/python3.8/site-packages/torch/nn/functional.py:1674:0
 ),
 4 defined in (%4 : int = prim::Constant[value=1](), scope: __module.dense # /Users/qqcao/.pyenv/versions/nlpnrg/lib/python3.8/site-packages/torch/nn/functional.py:1674:0
 ),
 4 defined in (%4 : int = prim::Constant[value=1](), scope: __module.dense # /Users/qqcao/.pyenv/versions/nlpnrg/lib/python3.8/site-packages/torch/nn/functional.py:1674:0
 )]

In [86]:
[i for i in dir(n) if not i.startswith('__')]

['addBlock',
 'addInput',
 'addOutput',
 'attributeNames',
 'blocks',
 'cconv',
 'copyAttributes',
 'destroy',
 'eraseOutput',
 'f',
 'f_',
 'findAllNodes',
 'findNode',
 'fs',
 'fs_',
 'g',
 'g_',
 'gs',
 'gs_',
 'hasAttribute',
 'hasAttributes',
 'hasMultipleOutputs',
 'hasUses',
 'i',
 'i_',
 'input',
 'inputs',
 'inputsAt',
 'insertAfter',
 'insertBefore',
 'is',
 'isNondeterministic',
 'is_',
 'kind',
 'kindOf',
 'moveAfter',
 'moveBefore',
 'mustBeNone',
 'output',
 'outputs',
 'outputsAt',
 'outputsSize',
 'pyname',
 'pyobj',
 'removeAllInputs',
 'removeAttribute',
 'removeInput',
 'replaceAllUsesWith',
 'replaceInput',
 'replaceInputWith',
 's',
 's_',
 'scalar_args',
 'scopeName',
 'sourceRange',
 'ss',
 'ss_',
 't',
 't_',
 'ts',
 'ts_',
 'z',
 'z_',
 'zs',
 'zs_']

In [82]:
n.s('name')

RuntimeError: required keyword attribute 'name' is undefined

In [85]:
n.pyobj()

RuntimeError: expected a prim::PythonOp but found a aten::addmm

In [87]:
{i:getattr(n, n.kindOf(i))(i) for i in n.attributeNames()}

{}

In [61]:
list(n.inputs())

[self.1 defined in (%self.1 : __torch__.BertIntermediate, %input : Float(32:128, 128:1) = prim::Param()
 )]

In [76]:
# {i.debugName(): i for i in fc_graph.nodes()}
# [[n for i in dir(n) if not i.startswith('__')] for n in fc_graph.nodes()]
nodes = list(fc_graph.nodes())
# [i for i in dir(n) if not i.startswith('__')]

%7 : Float(128:1, 512:128) = aten::t(%6), scope: __module.dense # /Users/qqcao/.pyenv/versions/nlpnrg/lib/python3.8/site-packages/torch/nn/functional.py:1674:0

In [79]:
n=nodes[5]

In [37]:
next(fc_graph.outputs()).debugName()

'19'

In [20]:
# from torchviz import make_dot, make_dot_from_trace
# fc_model.eval()
# with torch.onnx.select_model_mode_for_export(fc_model, False):
#     fc_viz_trace, _ = torch.jit._get_trace_graph(fc_model, args=(input_states,))
# # make_dot(fc_model(input_states), params=dict(fc_model.named_parameters()))
# make_dot_from_trace(fc_viz_trace)

In [ ]:
import torch.autograd.profiler as profiler

with profiler.profile() as prof:
    with profiler.record_function("model_inference"):
        model(inputs)

prof.export_chrome_trace("trace.json")

In [21]:
# trace.graph

In [19]:
# trace.graph

In [3]:
# trace.inlined_graph

In [21]:
# for in_node in trace.graph.inputs():
#     print(in_node)
# print('start output nodes')
# for in_node in trace.inlined_graph.nodes():
#     print(in_node)


In [5]:
# from torchviz import make_dot, make_dot_from_trace

In [6]:
# make_dot(model(inputs), params=dict(model.named_parameters()))

In [7]:
import warnings
from collections import namedtuple
from distutils.version import LooseVersion
from graphviz import Digraph
import torch
from torch.autograd import Variable

Node = namedtuple('Node', ('name', 'inputs', 'attr', 'op'))

def resize_graph(dot, size_per_element=0.15, min_size=12):
    """Resize the graph according to how much content it contains.
    Modify the graph in place.
    """
    # Get the approximate number of nodes and edges
    num_rows = len(dot.body)
    content_size = num_rows * size_per_element
    size = max(min_size, content_size)
    size_str = str(size) + "," + str(size)
    dot.graph_attr.update(size=size_str)

def replace(name, scope):
    return '/'.join([scope[name], name])

def parse(graph):
    scope = {}
    for n in graph.nodes():
        inputs = [i.debugName() for i in n.inputs()]
        for i in range(1, len(inputs)):
            scope[inputs[i]] = n.scopeName()

        uname = next(n.outputs()).debugName()
        if not n.scopeName():
            warnings.warn('{} has empty scope name'.format(n))
        scope[uname] = n.scopeName()
    scope['0'] = 'input'
    print('scope:', scope)
    nodes = []
    for n in graph.nodes():
        attrs = {k: n[k] for k in n.attributeNames()}
        attrs = str(attrs).replace("'", ' ')
        inputs = [replace(i.debugName(), scope) for i in n.inputs()]
        uname = next(n.outputs()).debugName()
        nodes.append(Node(**{'name': replace(uname, scope),
                             'op': n.kind(),
                             'inputs': inputs,
                             'attr': attrs}))

    for n in graph.inputs():
        uname = n.debugName()
        if uname not in scope.keys():
            scope[uname] = 'unused'
        nodes.append(Node(**{'name': replace(uname, scope),
                             'op': 'Parameter',
                             'inputs': [],
                             'attr': str(n.type())}))

    return nodes

def dot_from_graph(graph):
    # # from tensorboardX
    # if LooseVersion(torch.__version__) >= LooseVersion("0.4.1"):
    #     torch.onnx._optimize_trace(trace, torch._C._onnx.OperatorExportTypes.ONNX_ATEN_FALLBACK)
    # elif LooseVersion(torch.__version__) >= LooseVersion("0.4"):
    #     torch.onnx._optimize_trace(trace, False)
    # else:
    #     torch.onnx._optimize_trace(trace)
    # graph = trace.inlined_graph
    # list_of_nodes = parse(graph)

    node_attr = dict(style='filled',
                     shape='box',
                     align='left',
                     fontsize='12',
                     ranksep='0.1',
                     height='0.2')

    dot = Digraph(node_attr=node_attr, graph_attr=dict(size="12,12"))

    for node in graph.nodes:
        dot.node(node.name, label=node.name.replace('/', '\n'))
        if node.inputs:
            for inp in node.inputs:
                dot.edge(inp, node.name)

    resize_graph(dot)

    return dot

# make_dot_from_trace(trace)


In [22]:
# from torchprofile.utils import trace
# graph = trace(model, inputs)
# dot_from_graph(graph)

In [ ]:
import warnings
from collections import namedtuple
from distutils.version import LooseVersion
from graphviz import Digraph
import torch
from torch.autograd import Variable

Node = namedtuple('Node', ('name', 'inputs', 'attr', 'op'))

def resize_graph(dot, size_per_element=0.15, min_size=12):
    """Resize the graph according to how much content it contains.
    Modify the graph in place.
    """
    # Get the approximate number of nodes and edges
    num_rows = len(dot.body)
    content_size = num_rows * size_per_element
    size = max(min_size, content_size)
    size_str = str(size) + "," + str(size)
    dot.graph_attr.update(size=size_str)

def replace(name, scope):
    return '/'.join([scope[name], name])

def parse(graph):
    scope = {}
    for n in graph.nodes():
        inputs = [i.debugName() for i in n.inputs()]
        for i in range(1, len(inputs)):
            scope[inputs[i]] = n.scopeName()

        uname = next(n.outputs()).debugName()
        if not n.scopeName():
            warnings.warn('{} has empty scope name'.format(n))
        scope[uname] = n.scopeName()
    scope['0'] = 'input'
    print('scope:', scope)
    nodes = []
    for n in graph.nodes():
        attrs = {k: n[k] for k in n.attributeNames()}
        attrs = str(attrs).replace("'", ' ')
        inputs = [replace(i.debugName(), scope) for i in n.inputs()]
        uname = next(n.outputs()).debugName()
        nodes.append(Node(**{'name': replace(uname, scope),
                             'op': n.kind(),
                             'inputs': inputs,
                             'attr': attrs}))

    for n in graph.inputs():
        uname = n.debugName()
        if uname not in scope.keys():
            scope[uname] = 'unused'
        nodes.append(Node(**{'name': replace(uname, scope),
                             'op': 'Parameter',
                             'inputs': [],
                             'attr': str(n.type())}))

    return nodes

def make_dot_from_trace(trace):
    """ Produces graphs of torch.jit.trace outputs
    Example:
    >>> trace, = torch.jit.trace(model, args=(x,))
    >>> dot = make_dot_from_trace(trace)
    """
    # # from tensorboardX
    # if LooseVersion(torch.__version__) >= LooseVersion("0.4.1"):
    #     torch.onnx._optimize_trace(trace, torch._C._onnx.OperatorExportTypes.ONNX_ATEN_FALLBACK)
    # elif LooseVersion(torch.__version__) >= LooseVersion("0.4"):
    #     torch.onnx._optimize_trace(trace, False)
    # else:
    #     torch.onnx._optimize_trace(trace)
    graph = trace.inlined_graph
    list_of_nodes = parse(graph)

    node_attr = dict(style='filled',
                     shape='box',
                     align='left',
                     fontsize='12',
                     ranksep='0.1',
                     height='0.2')

    dot = Digraph(node_attr=node_attr, graph_attr=dict(size="12,12"))

    for node in list_of_nodes:
        dot.node(node.name, label=node.name.replace('/', '\n'))
        if node.inputs:
            for inp in node.inputs:
                dot.edge(inp, node.name)

    resize_graph(dot)

    return dot

make_dot_from_trace(trace)
